# Variable scope in list comprehension
Be careful about variable scope in list comprehension!

In [61]:
# Make sure there is no variable named "k".
try: 
    del k
except:
    pass 

print "Does variable 'k' exist?", 'k' in locals() or 'k' in globals()
print "List comprehension", [k for k in range(5)]

print "Does variable 'k' exist now?", "k" in locals(), k
# k was created in that list comprehension. 

# Conquenently, we may see this:
for i in range(3):
    a = [i for i in range(99)]
    print "Count", i

Does variable 'k' exist? False
List comprehension [0, 1, 2, 3, 4]
Does variable 'k' exist now? True 4
Count 98
Count 98
Count 98


**Remark.** In Python 3, variables in a list comprehension are local to the list, but avoiding a name conflict still makes codes more readable.

# Looping over objects

We can directly loop over iterable objects (e.g. lists) or a string-like object.

In [67]:
some_python_packages = ["komodo", "jellyfish", "frog", "pandas", "turtle", "conda"]

print "# Loop over a list with index. "
for i in range(len(some_python_packages)):
    print some_python_packages[i]

print "\n# Actually, we can just loop over the list."
for package in some_python_packages:
    print package

print "\n# Is a string iterable?", hasattr(a[1], '__iter__')
print "# We can loop over a string like "
for c in "python":
    print c
print "# because Python will create an iterator for objects with __getitem__"
print "# Does a string have __getitem__?", hasattr("I'm a string.", '__getitem__')

# Loop over a list with index. 
komodo
jellyfish
frog
pandas
turtle
conda

# Actually, we can just loop over the list.
komodo
jellyfish
frog
pandas
turtle
conda

# Is a string iterable? False
# We can loop over a string like 
p
y
t
h
o
n
# because Python will create an iterator for objects with __getitem__
# Does a string have __getitem__? True


### Enumerate
What if we need indexes also? Call enumerate function.

In [55]:
for i in range(len(some_python_packages)): 
    print (i, some_python_packages[i])

print "# Just enumerate it."
for i in enumerate(some_python_packages):
    print i

(0, 'komodo')
(1, 'jellyfish')
(2, 'frog')
(3, 'pandas')
(4, 'turtle')
(5, 'conda')
# Just enumerate it.
(0, 'komodo')
(1, 'jellyfish')
(2, 'frog')
(3, 'pandas')
(4, 'turtle')
(5, 'conda')


### Don't create a list unless it is nessary.

We only need "iter" operation, so just create an iterator.

In [53]:
def f(idx, s):
    print idx, s
    return s

print "# Create a new list and iterate over it. We create a list, but never use elements after 'pandas'"
for i in [f(j, k) for j, k in enumerate(some_python_packages) if j % 2]:
    if i == 'pandas': break
        
print "\n# Use iterators instead."
# Find the next element when for-loop requests.
for i in (f(j, r) for j, r in enumerate(some_python_packages) if j % 2):
    if i == 'pandas': break

# Also note the variable scope in an iterator!
print "\nDoes 'r' exist?", 'r' in locals() or 'r' in globals() 

# Create a new list and iterate over it. We create a list, but never use elements after 'pandas'
1 jellyfish
3 pandas
5 conda

# Use iterators instead.
1 jellyfish
3 pandas

Does 'r' exist? False


### Another example about looping over a dictionary

In [52]:
from sys import modules
package_dict = {package: package in modules for package in some_python_packages}

list1 = [i for i in package_dict.keys()]        # Create a list consisting of the keys of package_dict
list2 = [i for i in package_dict.iterkeys()]    # Create an iterator. This method has been removed in Python 3.
list3 = [i for i in package_dict]               # Simply iterate over the key set.

assert list1 == list2
assert list1 == list3

# Create a new list only if we really need it (e.g. a sorted list).
for i in sorted(a_dict.keys()):
    pass
    # Do something
   

# How to initialize a nested list

Let's create a 5*3 matrix, represented by a nested list [[0, 0, 0], [0, 0, 0], [0, 255, 0], [0, 0, 0], [0, 0, 0]].

In [68]:
a = [[0] * 3] * 5
print "First try:", a
# looks fine. Let's modify an element.
a[2][1] = 255
print "Oops!     ", a 
# because all elements point to one single list [0, 0, 0].

a = [[0] * 3 for _ in range(5)] 
a[2][1] = 255
print "Try again:", a
# Phew.

a = [b[:] for b in ([[0] * 3] * 5)]
a[2][1] = 255
print "Also works:", a
# Actually this is faster. 
# The previous method created five [0, 0, 0]. 
# This one created only one [0, 0, 0] and copy it 5 times.


First try: [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
Oops!      [[0, 255, 0], [0, 255, 0], [0, 255, 0], [0, 255, 0], [0, 255, 0]]
Try again: [[0, 0, 0], [0, 0, 0], [0, 255, 0], [0, 0, 0], [0, 0, 0]]
Also works: [[0, 255, 0], [0, 255, 0], [0, 255, 0], [0, 255, 0], [0, 255, 0]]


# Close a file

We can call "close" method to close a file object although this totally does not matter in Assignment 1. Actually, Python automatically do garbage collection (gc) if the opened file is EOF or a Python kernel/shell close. However, there are some reasons why we should always close a file object if we no longer need it.

1. It is unpredictable when gc is done, and unclosed file objects will keep wasting system resources. It could turn to be something scary if a script runs for a long time.

2. It depends on the version of Python interpreters when file objects are closed. Closing file objects make codes more portable.

3. This could be a serious problem when writing data to a file. For example,

In [ ]:
f = open("output.txt", "w")  # data.txt is created
f.write("data")              # data is in buffer, and "output.txt is still empty.

# Do something and make your computer crash.

f.close()                    # Modify output.txt # It's too late.

Alternatively, we can "with" statement to open/close files in a more pythonic way. The file object will be automatically closed when we exit from the "with" block.

In [ ]:
with open("data.txt", 'w') as f:
    # Do something
    pass

#Open multiple files
with open("data.txt", 'r') as data, open("output.txt", 'w') as output:
    # Do something
    pass